In [ ]:
!pip install plotly pandas numpy

# Streamlit tutorial

## Covid19 dataset

In this tutorial we will be using covid19 data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series. 

In [ ]:
import pandas as pd

confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths    = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [ ]:
confirmed

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,40287,40357,40510,40626,40687,40768,40833,40937,41032,41145,41268,41334,41425,41501,41633,41728,41814,41935,41975,42033,42092,42297,42463,42609,42795,42969,43035,43240,43403,43628,43851,44228,44443,44503,44706,44988,45280,45490,45716,45839
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17350,17651,17948,18250,18556,18858,19157,19445,19729,20040,20315,20634,20875,21202,21523,21904,22300,22721,23210,23705,24206,24731,25294,25801,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196,32761,33556,34300,34944,35600,36245
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,54616,54829,55081,55357,55630,55880,56143,56419,56706,57026,57332,57651,57942,58272,58574,58979,59527,60169,60800,61381,62051,62693,63446,64257,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774,74862,75867,77000,78025,79110,80168
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3623,3623,3811,3811,4038,4038,4038,4325,4410,4517,4567,4665,4756,4825,4888,4910,5045,5135,5135,5319,5383,5437,5477,5567,5616,5725,5725,5872,5914,5951,6018,6066,6142,6207,6256,6304,6351,6428,6534,6610
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7829,8049,8338,8582,8829,9026,9381,9644,9871,10074,10269,10558,10805,11035,11228,11577,11813,12102,12223,12335,12433,12680,12816,12953,13053,13228,13374,13451,13615,13818,13922,14134,14267,14413,14493,14634,14742,14821,14920,15008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,47616,48129,48628,49134,49579,49989,50442,50952,51528,51948,52571,53075,53520,54060,54775,55408,56090,56672,57226,57657,58158,58838,59422,60065,60784,61514,62167,63031,63867,64935,66186,67296,68768,70254,71644,73196,75007,76727,78493,80429
267,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
268,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2056,2057,2057,2057,2060,2060,2060,2060,2060,2061,2062,2062,2063,2063,2063,2063,2063,2063,2067,2070,2070,2071,2071,2071,2071,2072,2072,2072,2078,2081,2083,2086,2090,2093,2099,2107,2114,2124,2137,2148
269,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15897,15982,16000,16035,16095,16117,16117,16200,16243,16285,16325,16415,16432,16480,16543,16661,16698,16770,16819,16908,16954,16971,16997,17036,17056,17093,17097,17123,17187,17243,17280,17350,17373,17394,17424,17454,17466,17535,17553,17569


Now we preprocess the data.

In [ ]:
date_columns = [x for x in confirmed][4:]

confirmed = confirmed.groupby(['Country/Region'])[date_columns].sum().reset_index()
deaths    = deaths.groupby(['Country/Region'])[date_columns].sum().reset_index()
recovered = recovered.groupby(['Country/Region'])[date_columns].sum().reset_index()

countries = confirmed['Country/Region'].to_list()

confirmed.rename(columns={'Country/Region':'Country'}, inplace=True)
deaths.rename(columns={'Country/Region':'Country'}, inplace=True)
recovered.rename(columns={'Country/Region':'Country'}, inplace=True)

confirmed = confirmed.set_index(confirmed.Country)[date_columns].T
deaths = deaths.set_index(deaths.Country)[date_columns].T
recovered = recovered.set_index(recovered.Country)[date_columns].T
countries = [x for x in confirmed]

deaths

Country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo (Brazzaville),...,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
1/25/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/26/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/27/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,82,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/28/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,131,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/29/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11/23/20,1695,716,2294,76,337,4,37122,1976,907,2459,1160,163,339,6416,7,1104,15755,120,43,0,8916,2342,31,169485,3,3069,68,1765,1,104,0,435,11570,63,101,15106,4742,35479,7,93,...,0,74,28,693,1097,0,113,20968,60,43131,90,1197,116,6406,4222,380,7,86,21,60,0,64,115,2862,12511,257779,181,11423,554,55327,71,604,0,873,35,645,1,609,357,273
11/24/20,1712,735,2309,76,338,4,37432,2002,907,2577,1194,163,340,6448,7,1112,15938,125,43,0,8928,2394,31,170115,3,3226,68,1784,1,104,0,436,11676,63,101,15131,4742,35677,7,93,...,0,74,28,709,1156,0,113,21083,60,43668,94,1202,117,6500,4308,385,7,86,21,60,0,64,115,2935,12672,259925,186,11619,559,55935,72,604,0,876,35,656,1,609,357,274
11/25/20,1725,743,2329,76,340,4,37714,2040,907,2667,1224,163,340,6487,7,1119,16077,129,43,0,8933,2429,31,170769,3,3367,68,1810,1,104,0,437,11733,63,101,15138,4742,35860,7,93,...,0,74,28,732,1199,0,113,21201,61,44037,96,1210,117,6555,4393,391,7,86,21,60,0,64,116,2983,12840,262222,191,11857,563,56630,73,606,0,880,35,665,1,611,357,274
11/26/20,1737,753,2352,76,341,4,37941,2068,907,2773,1256,163,341,6524,7,1128,16219,131,43,0,8939,2480,31,171460,3,3529,68,1846,1,105,0,437,11818,63,101,15235,4742,36019,7,93,...,0,74,28,749,1245,0,113,21289,61,44374,99,1215,117,6622,4509,399,7,86,21,60,0,64,116,3034,13014,263454,191,12091,564,57128,74,607,0,884,35,680,1,612,357,274


Now we can easily select data between dates for selected countries.

In [ ]:
confirmed.loc['3/1/20':'11/27/20', ['Czechia','Austria']]

Country,Czechia,Austria
3/1/20,3,14
3/2/20,3,18
3/3/20,5,21
3/4/20,8,29
3/5/20,12,41
...,...,...
11/23/20,496638,250333
11/24/20,502534,254710
11/25/20,505215,260512
11/26/20,511520,266038


And plot selected data.

In [ ]:
import plotly.graph_objects as go

fig=go.Figure()


for x in ['Czechia','Austria']:
    fig.add_trace(
        go.Scatter(
            y=confirmed.loc['3/1/20':'11/27/20', x].to_list(),
            x=date_columns, 
            name=x+' confirmed'
        )
    )
    fig.add_trace(
        go.Scatter(
            y=deaths.loc['3/1/20':'11/27/20', x].to_list(),
            x=date_columns, 
            name=x+' deaths'
        )
    )
    fig.add_trace(
        go.Scatter(
            y=recovered.loc['3/1/20':'11/27/20', x].to_list(),
            x=date_columns, 
            name=x+' recovered'
        )
    )


fig.update_layout(
    title="Number of COVID19 confirmed cases, deaths and recovered", 
    xaxis_title="Date",
    yaxis_title="Cases (logarithmic scale)",
    yaxis_type="log", # switch log scale on for y axis
    hovermode='x' # compare data on hoover by default
  )



It would be convenient to easily pick dates, select countries and other parameters. So is it possible to create web app for data visualisation? 

Yes, with streamlit.

## streamlit

Streamlit is an open-source Python library that makes it easy to create and share beautiful, custom web apps for machine learning and data science. In just a few minutes you can build and deploy powerful data apps - as I will show you below.

API Documentation link: https://docs.streamlit.io/en/stable/api.html

Let's write simple file on disk.

In [ ]:
%%writefile streamlitapp.py

import streamlit as st

st.write("Hello world!")

Overwriting streamlitapp.py


You can now run streamlit server with command `streamlit run streamlitapp.py` and it should look like this:

(Note: you will not be able to access the app in colab) 

![hello_world](https://raw.githubusercontent.com/zombak79/streamlit-tutorial/main/img/hello_world.gif)

Streamlit have also many interactive widgets:

In [ ]:
%%writefile streamlitapp.py

import streamlit as st

st.title('Demo app')

st.write("Let's test some **interactive widgets:**")

# ****************** BUTTONS **************************
st.write("**Buttons**")
if st.button('Button 1', key='button_1'):
     st.write(':-)')
else:
     st.write(':-(')
                
if st.button('Button 2', key='button_2'):
     st.write(':-)')
else:
     st.write(':-(')
        
# ****************** CHECKBOX *************************
st.write("**Checkbox**")
check = st.checkbox("Check me", value=False, key='check_1')
if check:
    st.write('Checked')
else:
    st.write('Not checked')
    
# ******************** RADIO **************************
st.write("**Radio button**")
radio = st.radio(
    "What is your favorite season?",
    options=['Spring', 'Summer', 'Autumn', 'Winter'],
    key='radio_1'    
)
st.write(radio)

# ****************** SELECT BOX ***********************
st.write("**Select box**")
sbox = st.selectbox(
    "What is your favorite season?",
    options=['Spring', 'Summer', 'Autumn', 'Winter'],
    key='sbox_1'
)
st.write(sbox)

# ****************** MULTI SELECT *********************
st.write("**Select box**")
ms = st.multiselect(
    "What is your favorite season?",
    options=['Spring', 'Summer', 'Autumn', 'Winter'],
    key='ms_1'
)
st.write(ms)

# ****************** SLIDER ***************************
st.write("**Slider**")
slider = st.slider(
    "Select number",
    min_value = 0,
    max_value = 10,
    step = 1,
    key="slider_1"
)
st.write("Number selected", slider)
st.write("*Note: for picking from list, you can use* `st.select_slider`*. See documentation for more info.*")

# **************** INPUTS *****************************
st.write("**Text Input**")
some_text = st.text_input("Input text:", key="inp_1")
st.write(some_text)
st.write("*Note: same for numbers (number_input), long texts (textarea), date and time inputs.")


Overwriting streamlitapp.py


![widgets](https://raw.githubusercontent.com/zombak79/streamlit-tutorial/main/img/widgets.gif)

We can arrange objects on page in columns with `st.beta_columns()`.

We can also display running code with `st.echo()`.

In [ ]:
%%writefile streamlitapp.py

import streamlit as st

st.title('Demo app')

st.write("Let's test some **interactive widgets:**")

# ****************** BUTTONS **************************

with st.beta_container():
    st.write("**Buttons**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        if col1.button('Button 1', key='button_1'):
             col2.write(':-)')
        else:
             col2.write(':-(')

        col1, col2 = st.beta_columns(2)
        if col1.button('Button 2', key='button_2'):
             col2.write(':-)')
        else:
             col2.write(':-(')
                          
# ****************** CHECKBOX *************************
with st.beta_container():
    st.write("**Checkbox**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        check = col1.checkbox("Check me", value=False, key='check_1')
        if check:
            col2.write('Checked')
        else:
            col2.write('Not checked')
    
# ******************** RADIO **************************
with st.beta_container():
    st.write("**Radio button**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        radio = col1.radio(
            "What is your favorite season?",
            options=['Spring', 'Summer', 'Autumn', 'Winter'],
            key='radio_1'    
        )
        col2.write(radio)

# ****************** SELECT BOX ***********************
with st.beta_container():
    st.write("**Select box**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        sbox = col1.selectbox(
            "What is your favorite season?",
            options=['Spring', 'Summer', 'Autumn', 'Winter'],
            key='sbox_1'
        )
        col2.write(sbox)

# ****************** MULTI SELECT *********************
with st.beta_container():
    st.write("**Multi select box**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        ms = col1.multiselect(
            "What is your favorite season?",
            options=['Spring', 'Summer', 'Autumn', 'Winter'],
            key='ms_1'
        )
        col2.write(ms)

# ****************** SLIDER ***************************
with st.beta_container():
    st.write("**Slider**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        slider = col1.slider(
            "Select number",
            min_value = 0,
            max_value = 10,
            step = 1,
            key="slider_1"
        )
        # multiple inputs work only for st.write and st.sidebar.write
        col2.write("Number selected " + str(slider)) 
    st.write("*Note: for picking from list, you can use* `st.select_slider`*. See documentation for more info.*")

# **************** INPUTS *****************************
with st.beta_container():
    st.write("**Text Input**")
    with st.echo():
        col1, col2 = st.beta_columns(2)
        some_text = col1.text_input("Input text:", key="inp_1")
        col2.write(some_text)
    st.write("*Note: same for numbers (number_input), long texts (textarea), date and time inputs.*")



Overwriting streamlitapp.py


![columns_echo.gif](https://raw.githubusercontent.com/zombak79/streamlit-tutorial/main/img/columns_echo.gif)

With `st.sidebar` we can separate widgets for user input and visualised data.

In [ ]:
%%writefile streamlitapp.py

import streamlit as st

st.set_page_config(
    page_title='Demo app', 
    page_icon=None, 
    layout='centered', 
    initial_sidebar_state='expanded')


st.title('Demo app')
st.sidebar.title("Parameters")
st.write("Let's test some **interactive widgets:**")

# ****************** MULTI SELECT *********************
st.write("**Select box**")
ms = st.sidebar.multiselect(
    "What is your favorite season?",
    options=['Spring', 'Summer', 'Autumn', 'Winter'],
    key='ms_1'
)
st.write(ms)



Overwriting streamlitapp.py


![sidebar.gif](https://raw.githubusercontent.com/zombak79/streamlit-tutorial/main/img/sidebar.gif)

## Lets put it all together

We will create two functions. First will download the data and the second will create plotly visualization. We will also cache them with `@st.cache` to save time.

Widgets will be in the sidebar.

In [ ]:
%%writefile streamlitapp.py

import pandas as pd
import plotly.graph_objects as go
import streamlit as st
from datetime import datetime

st.set_page_config(
    page_title='Covid19 data explorer', 
    page_icon=None, 
    layout='centered', 
    initial_sidebar_state='expanded')

@st.cache
def load_data(date):
    confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    deaths    = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
    recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
    date_columns = [x for x in confirmed][4:]

    confirmed = confirmed.groupby(['Country/Region'])[date_columns].sum().reset_index()
    deaths    = deaths.groupby(['Country/Region'])[date_columns].sum().reset_index()
    recovered = recovered.groupby(['Country/Region'])[date_columns].sum().reset_index()

    countries = confirmed['Country/Region'].to_list()

    confirmed.rename(columns={'Country/Region':'Country'}, inplace=True)
    deaths.rename(columns={'Country/Region':'Country'}, inplace=True)
    recovered.rename(columns={'Country/Region':'Country'}, inplace=True)

    confirmed = confirmed.set_index(confirmed.Country)[date_columns].T
    deaths = deaths.set_index(deaths.Country)[date_columns].T
    recovered = recovered.set_index(recovered.Country)[date_columns].T
    countries = [x for x in confirmed]

    return countries, date_columns, confirmed, recovered, deaths

@st.cache
def get_plotly_object(selected_countries, date_from, date_to, show_confirmed, show_deaths, show_recovered, show_legend, logaritmic):
    date_from = date_from.strftime('%-m/%-d/%y')
    date_to = date_to.strftime('%-m/%-d/%y')
    
    fig=go.Figure()


    for x in selected_countries:
        
        if show_confirmed:
            fig.add_trace(
                go.Scatter(
                    y=confirmed.loc[date_from:date_to, x].to_list(),
                    x=dates[dates.index(date_from):dates.index(date_to)],
                    name=x+' confirmed'
                )
            )
        
        if show_deaths:
            fig.add_trace(
                go.Scatter(
                    y=deaths.loc[date_from:date_to, x].to_list(),
                    x=dates[dates.index(date_from):dates.index(date_to)], 
                    name=x+' deaths'
                )
            )
        
        if show_recovered:
            fig.add_trace(
                go.Scatter(
                    y=recovered.loc[date_from:date_to, x].to_list(),
                    x=dates[dates.index(date_from):dates.index(date_to)], 
                    name=x+' recovered'
                )
            )
    
    if logaritmic:
        ya = 'log'
    else:
        ya = 'linear'
        
    fig.update_layout(
        title="Number of COVID19 confirmed cases, deaths and recovered", 
        xaxis_title="Date",
        yaxis_title="Cases (logarithmic scale)",
        yaxis_type=ya, # switch log scale on for y axis
        hovermode='x', # compare data on hoover by default
        showlegend=show_legend
      )
    
    return fig


countries, dates, confirmed, recovered, deaths = load_data(datetime.today().strftime('%Y-%m-%d'))

st.title('Covid19 data explorer')

# sidebar
selected_countries = st.sidebar.multiselect(
    "Select countries",
    options=countries,
    default='Czechia',
    key='ms_1'
)

date_from = st.sidebar.date_input(
    "Date from",
    min_value = datetime.strptime(dates[0], '%m/%d/%y'),
    max_value = datetime.strptime(dates[-1], '%m/%d/%y'),
    value=datetime.strptime(dates[0], '%m/%d/%y'),
    key="date_from"
)

date_to = st.sidebar.date_input(
    "Date to",
    min_value = datetime.strptime(dates[0], '%m/%d/%y'),
    max_value = datetime.strptime(dates[-1], '%m/%d/%y'),
    value=datetime.strptime(dates[-1], '%m/%d/%y'),
    key="date_to"
)

show_confirmed = st.sidebar.checkbox("Show confirmed cases", value=True, key='check_1')
show_deaths = st.sidebar.checkbox("Show deaths", value=True, key='check_2')
show_recovered = st.sidebar.checkbox("Show recovered", value=True, key='check_3')
show_legend = st.sidebar.checkbox("Show legend", value=True, key='check_4')
logaritmic = st.sidebar.checkbox("Log scale", value=True, key='check_5')

# use data to generate plot

plotly_fig = get_plotly_object(selected_countries, date_from, date_to,show_confirmed,show_deaths,show_recovered,show_legend,logaritmic)
st.write(plotly_fig)

Overwriting streamlitapp.py


![covid.gif](https://raw.githubusercontent.com/zombak79/streamlit-tutorial/main/img/covid.gif)